Домашняя работа №3 по курсу "Продвинутое машинное обучение"
выполнил Денисов Артем MADE-ML-22

In [243]:
import pickle
from pprint import pprint
from typing import List, Dict
from random import shuffle, seed, choices, random

import numpy as np
import pandas as pd

1. Реализуйте базовый частотный метод по Шерлоку Холмсу
○	подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
○	возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
○	расшифруйте их таким частотным методом.


In [3]:
ALPHABET = 'абвгдежзийклмнопрстуфхцчшщъыьэюя' + ' '

In [4]:
def open_text_file(filepath):
    text = []
    with open(filepath, encoding='utf-8') as file:
        for row in file:
            text.append(row)
    return ''.join(text)

In [202]:
def preprocess_text(text, alphabet):
    new_text = []
    for letter in text.lower():
        if letter in alphabet:
            new_text.append(letter)
    return ''.join(new_text)

In [203]:
def build_letter_frequency(text, alphabet):
    frequency_dict = {letter: 0 for letter in alphabet}
    for letter in text:
        frequency_dict[letter] += 1
    return frequency_dict

In [206]:
corpus = open_text_file('WarAndPeace.txt')
processed_corpus = preprocess_text(corpus, ALPHABET)
train_frequency = build_letter_frequency(processed_corpus, ALPHABET)
len(corpus), len(processed_corpus)

(717873, 649364)

In [17]:
def shuffle_text(text, alphabet):
    seed(42)
    shuffled_text = []
    
    shuffled_alphabet_list = [letter for letter in alphabet]
    alphabet_list = [letter for letter in alphabet]
    shuffle(shuffled_alphabet_list)
    shuffle_mapping = {a: b for a, b in zip(alphabet_list, shuffled_alphabet_list)} 
    for letter in text:
        if letter in alphabet:
            shuffled_text.append(shuffle_mapping[letter])
        else:
            shuffled_text.append(letter)
    return ''.join(shuffled_text), shuffle_mapping

In [209]:
test_text = "«Мне теперь так ясна моя мысль, – говорил Толстой в 1877 году, завершая работу над романом. – Так, в „Анне Карениной“ я люблю мысль семейную, в „Войне и мире“ любил мысль народную вследствие войны 1812 года…».[25] Конечно, и в «Войне и мире» есть «семейная хроника», и в «Анне Карениной» есть «картины народной жизни». Но для каждого романа нужна была своя, особенная любовь. «Мысль семейная» для 70-х годов XIX века особенно актуальна. «Одна из самых важных задач в этом текущем, для меня, например, – признавался Ф.М. Достоевский, – молодое поколение, и вместе с тем современная русская семья, которая, я предчувствую это, далеко не такова, как всего еще двадцать лет назад».[26]"
test_text = preprocess_text(test_text, ALPHABET)
test_text

'мне теперь так ясна моя мысль  говорил толстой в  году завершая работу над романом  так в анне карениной я люблю мысль семейную в войне и мире любил мысль народную вследствие войны  года конечно и в войне и мире есть семейная хроника и в анне карениной есть картины народной жизни но для каждого романа нужна была своя особенная любовь мысль семейная для х годов  века особенно актуальна одна из самых важных задач в этом текущем для меня например  признавался фм достоевский  молодое поколение и вместе с тем современная русская семья которая я предчувствую это далеко не такова как всего еще двадцать лет назад'

In [210]:
shuffled_text, shuffle_mapping = shuffle_text(test_text, ALPHABET)
shuffled_text

'йэмзямжмщ зяыузбаэызйшбзйьац ззршлшщкцзяшцаяшозлззршънзпылмщгыбзщыешянзэыъзщшйыэшйззяыузлзыээмзуыщмэкэшозбзцчецчзйьац замймоэнчзлзлшоэмзкзйкщмзцчекцзйьац зэыщшъэнчзлацмъаялкмзлшоэьззршъызушэмхэшзкзлзлшоэмзкзйкщмзмая замймоэыбзвщшэкуызкзлзыээмзуыщмэкэшозмая зуыщякэьзэыщшъэшозфкпэкзэшзъцбзуыфъшршзщшйыэызэнфэызеьцызалшбзшашемээыбзцчешл зйьац замймоэыбзъцбзвзршъшлззлмуызшашемээшзыуяныц эызшъэызкпзаыйьвзлыфэьвзпыъыхзлзияшйзямунюмйзъцбзймэбзэыжщкймщззжщкпэылыцабзтйзъшаяшмлаукоззйшцшъшмзжшушцмэкмзкзлймаямзазямйзашлщмймээыбзщнаауыбзамй бзушяшщыбзбзжщмъхнлаялнчзияшзъыцмушзэмзяыушлызуыузламршзмюмзълыъсыя зцмязэыпыъ'

In [211]:
def make_decoding_mapping(train_frequency, test_frequency):
    train_frequency_list = [(value, key) for key, value in train_frequency.items()]
    train_frequency_list.sort(reverse=True)
    test_frequency_list = [(value, key) for key, value in test_frequency.items()]
    test_frequency_list.sort(reverse=True)
    decoding_mapping = dict()
    for i in range(len(test_frequency_list)):
        train_counter, train_letter = train_frequency_list[i]
        test_counter, test_letter = test_frequency_list[i]
        decoding_mapping[test_letter] = train_letter
    return decoding_mapping

In [212]:
def decode_text(text, mapping):
    decoded_text = []
    for letter in text:
        decoded_text.append(mapping[letter])
    return ''.join(decoded_text)

In [213]:
test_frequency = build_letter_frequency(shuffled_text, ALPHABET)
decode_mapping = make_decoding_mapping(train_frequency, test_frequency)
print(decode_mapping)

{'з': ' ', 'ш': 'о', 'ы': 'а', 'м': 'е', 'э': 'и', 'а': 'н', 'л': 'т', 'й': 'с', 'я': 'л', 'ц': 'в', 'щ': 'р', 'к': 'к', 'ъ': 'д', 'у': 'м', 'б': 'у', 'о': 'п', ' ': 'я', 'н': 'г', 'ь': 'ь', 'ч': 'ы', 'е': 'з', 'р': 'б', 'п': 'ч', 'ж': 'й', 'ф': 'ж', 'в': 'ш', 'х': 'х', 'ю': 'ю', 'и': 'ц', 'т': 'э', 'с': 'щ', 'г': 'ф', 'д': 'ъ'}


In [214]:
decoded_text = decode_text(shuffled_text, decode_mapping)
print(decoded_text)

сие лейеря лам униа соу сьнвя  боторкв ловнлоп т  бодг чатерфау разолг иад росаиос  лам т аиие мареикиоп у вызвы сьнвя несепигы т топие к скре вызкв сьнвя иародигы тнведнлтке топиь  бода моиехио к т топие к скре енля несепиау шроикма к т аиие мареикиоп енля марлкиь иародиоп жкчик ио дву маждобо росаиа игжиа зьва нтоу онозеииау вызотя сьнвя несепиау дву ш бодот  тема онозеиио амлгавяиа одиа кч насьш тажиьш чадах т цлос лемгюес дву сеиу иайрксер  йркчиатавну эс донлоетнмкп  соводое йомовеике к тсенле н лес нотресеииау ргннмау несяу молорау у йредхгтнлтгы цло давемо ие ламота мам тнебо еюе дтадщаля вел иачад


In [215]:
test_text

'мне теперь так ясна моя мысль  говорил толстой в  году завершая работу над романом  так в анне карениной я люблю мысль семейную в войне и мире любил мысль народную вследствие войны  года конечно и в войне и мире есть семейная хроника и в анне карениной есть картины народной жизни но для каждого романа нужна была своя особенная любовь мысль семейная для х годов  века особенно актуальна одна из самых важных задач в этом текущем для меня например  признавался фм достоевский  молодое поколение и вместе с тем современная русская семья которая я предчувствую это далеко не такова как всего еще двадцать лет назад'

In [216]:
def letter_accuracy(true_text, decoded_text, alphabet):
    all_answers = 0
    correct_answers = 0
    for letter_true, letter_decoded in zip(true_text, decoded_text):
        if letter_decoded in alphabet:
            if letter_true == letter_decoded:
                correct_answers += 1
            all_answers += 1
    return correct_answers / all_answers

In [217]:
print(f"{letter_accuracy(test_text, decoded_text, ALPHABET):.3f}")

0.487


In [67]:
# test_test = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [218]:
def build_letter_frequency_bigram(text, n=2):
    frequency_dict = dict()
    for letter_index in range(len(text) - n + 1):
        bigram = text[letter_index:letter_index + n]
        if bigram in frequency_dict.keys():
            frequency_dict[bigram] += 1
        else:
            frequency_dict[bigram] = 1
    return frequency_dict

In [224]:
def decode_text_bigram(text, mapping, n=2):
    decoded_text = []
    for letter_index in range(len(text) - n + 1):
        bigram = text[letter_index:letter_index + n]
        decoded_text.append(mapping[bigram][0])
    decoded_text.append(mapping[bigram][1])
    return ''.join(decoded_text)

In [225]:
train_frequency_bigram = build_letter_frequency_bigram(text)
test_frequency_bigram = build_letter_frequency_bigram(shuffled_text)
len(train_frequency_bigram), len(test_frequency_bigram)

(263, 199)

In [226]:
decode_mapping = make_decoding_mapping(train_frequency_bigram, test_frequency_bigram)
decoded_text_bigram = decode_text_bigram(shuffled_text, decode_mapping)

In [227]:
decoded_text_bigram

'шъдж–щчшящждшщгххэъщтщ,щйыжхщщжхцлщыфщжщъшщщщфщйщжхщ цйщюхшяцъ,йю[йь»цюэщ„йчщущэщйщждшщщйсщэъдщлшчхэхэщфг,жхъцххщйыжхщэ.пйцзэщхщйщлщзъдюгщтхчджхъцфщщйыжхщюэшчщьэщхщтжфц„щ»щфдщлщзяющжхщыъщхъъъюэшюгщйщлщзъдюгщтхчдйцщгщэ.пйцзэъ,йхчъэшлъюгщйсщэъдщлшчхэхэщфйцщгщщлшяйхяююэшчщьэщфчыфцэгюэшвыъ,щлюыщяхшйчщущэъюэъчэъчцзюъэхлщ,оъбщйхээъ,жхъйцшщщйыжхщэ.пйцзэъ,выъ,йбжхщщцйщщхчлъоъбщйхээшсшы»ъэххэъощьэъюфъэхайюбщфючяюбйщщыгьщййлщщйж–чыъ“пйвыъ,щйхи,юэ«гышйшящйгыфцэюфэшх,чыйвщъщщщщтйышфщщтъющщщдйчяхъфхэфдюгщщйцщ–дэхж–пйэбцщчпйхээъ,йяъхйлъ,э.пшх,щхьщщюъ,г,йгчц1эътщ»щщхйлщшвыэфчхшюъдждшхцфъщлшщщт.щхшйъ“дв…фщ.ыжгщжфъиюэгщщч'

In [228]:
alphabet = ''.join(set(decoded_text_bigram))
print(f"{letter_accuracy(test_text, decoded_text_bigram, alphabet):.3f}")

0.010


Биграммы - это марковская цепь
из корпуса получаю вероятность перехода из буквы в букву

In [229]:
def estimate_probabilites(corpus, alphabet, n_gram=2):
    n_text = len(corpus)
    prob_dict = {letter: dict() for letter in alphabet}
    for i in range(n_text - n_gram):
        letter = corpus[i]
        next_letter = corpus[i + 1]
        if letter in alphabet and next_letter in alphabet:
            if next_letter in prob_dict[letter].keys():
                prob_dict[letter][next_letter] += 1
            else:
                prob_dict[letter][next_letter] = 1
                
    for letter in alphabet:
        letter_sum_probability = sum(prob_dict[letter].values())
        for next_letter in prob_dict[letter].keys():
            prob_dict[letter][next_letter] /= letter_sum_probability
            
    for letter in alphabet:
        letter_sum_probability = sum(prob_dict[letter].values())
        for next_letter in prob_dict[letter].keys():
            prob_dict[letter][next_letter] /= n_text
            
    return prob_dict

In [252]:
def count_probability(text, prob_dict, ngram=2):
    n_text = len(text)
    log_prob = 0
    for i in range(n_text - n_gram):
        n_letters = text[i:i + ngram]
        if n_letters in prob_dict.keys():
            log_prob += np.log(prob_dict[n_letters])
        else:
            log_prob += -10000
    return log_prob

In [253]:
prob_dict = estimate_probabilites(processed_corpus, ALPHABET)

In [258]:
def decode_text_mcmm(text, prob_dict, alphabet, n_steps=1000):
    decoded_text = ''
    current_log_proba = count_probability(text, prob_dict)
    mapping_for_decoding = {letter: letter for letter in alphabet}
    accepted_steps = 0
    for i in range(n_steps):
        letter_one, letter_two = choices(alphabet, k=2)
        test_mapping = mapping_for_decoding.copy()
        test_mapping[letter_one], test_mapping[letter_two] = test_mapping[letter_two], test_mapping[letter_one]
        try_decoded_text = decode_text(text, test_mapping)
        new_prob = count_probability(try_decoded_text, prob_dict)
        
        prob_condition = np.exp(new_prob - current_log_proba)
        if prob_condition > random():
            accepted_steps += 1
            mapping_for_decoding = test_mapping
            current_log_proba = new_prob
    return mapping_for_decoding, current_log_proba, accepted_steps / n_steps

In [260]:
best_proba = -20000
for i in range(20):
    mapping, proba, accepted = decode_text_mcmm(shuffled_text, prob_dict, ALPHABET, n_steps=15000)
    print(i, proba, accepted)
    if proba > best_proba:
        best_proba = proba
        best_mapping = mapping
        decoded_text = decode_text(shuffled_text, best_mapping)    
        print(decoded_text)
        accuracy = letter_accuracy(test_text, decoded_text, ALPHABET)
        print(f"{accuracy:.3f}")

<ipython-input-258-e43fa0bdfdcf>:13: RuntimeWarning: overflow encountered in exp
  prob_condition = np.exp(new_prob - current_log_proba)


0 -10091.692553649238 0.037533333333333335
нр ет г дьетомешсроенашенусвьеещаладиветавстафелеещазбепол дцошедохатберозеданоранеетомелеорр емод рирафешевыхвыенусвьес н фрбыелелафр еиенид евыхивенусвьеродазрбыелсв зстли елафруеещазоемар яраеиелелафр еиенид е стьес н фрошеждаримоеиелеорр емод рирафе стьемодтируеродазрафекиприераезвшемокзащаеданороербкроехувоеслашеасах ррошевыхальенусвьес н фрошезвшежещазалеел моеасах рраеомтбовьроеазроеипесонужелокружепозояелечтанет мбэ незвшен ршерогдин деегдипроловсшейнезаста лсмифеенаваза егамав ри еиелн ст есет несалд н ррошедбссмошес ньшематадошешегд зяблстлбыечтаезов маер етомалоемомелс щае э езлозюотьев теропоз
0.134
1 -9716.86373979716 0.0396
мне теперь так ясна моя мысль  говорил толстой в  году завершая работу над романом  так в анне карениной я люблю мысль семейную в войне и мире любил мысль народную вследствие войны  года конехно и в войне и мире есть семейная чроника и в анне карениной есть картины народной жизни но для каждого романа нужна бы

Триграммы